In [144]:
import numpy as np
import pandas as pd
import datetime
import os

In [193]:
# def process(file):
    
#     temp = pd.read_csv(file)
#     temp = temp[temp['Country/Region'] == 'US'] if 'Country/Region' in temp.columns else temp[temp['Country_Region'] == 'US']
    
#     if 'Admin2' in temp.columns:
#         temp['County'] = temp.Admin2
#         temp['State'] = temp.Province_State
#     elif temp['Province/State'].str.contains(',').any() and (temp['Province/State'] != 'Virgin Islands, U.S.').all():
#         x = temp['Province/State'].str.replace(' (From Diamond Princess)','', regex=False).replace('Unassigned Location','', regex=False).str.split(', ')
#         temp['County'] = x.map(lambda x: x[0])
#         temp['State'] = x.map(lambda x: x[-1])
#     else:
#         temp['County'] = ''
#         temp['State'] = temp['Province/State'].str.replace(' (From Diamond Princess)','', regex=False).str.replace('Unassigned Location','', regex=False)
    
#     temp['County'] = temp['County'].str.replace(' County', '', regex=False)
#     temp['Date'] = pd.to_datetime(file[-14:-4]).strftime('%Y-%m-%d')
    
#     temp['Lat'] = temp['Latitude'] if 'Latitude' in temp.columns else np.nan
    
#     if 'Longitude' in temp.columns:
#         temp['Long'] = temp['Longitude']
#     elif 'Long_' in temp.columns:
#         temp['Long'] = temp['Long_']
#     else:
#         temp['Long'] = np.nan
    
#     temp['Deaths'] = temp.Deaths
#     temp['Recovered'] = temp.Recovered if 'Recovered' in temp.columns else 0
#     temp['Active'] = temp['Active'] if 'Active' in temp.columns else 0
#     temp['FIPS'] = temp['FIPS'] if 'FIPS' in temp.columns else np.nan
    
#     temp = temp[['FIPS', 'County', 'State', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active']]        

#     return temp

In [337]:
# path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'

# if not os.path.exists('covid_19_US.csv'):
#     files = [i for i in os.listdir(path) if 'csv' in i]
#     data = pd.DataFrame([])

#     for i in files:
#         print('loading {}'.format(i))
#         temp = process(path + i)
#         data = pd.concat([data, temp], ignore_index=True)

#     print(data.shape)
# else:
#     data = pd.read_csv('covid_19_US.csv').drop('Unnamed: 0', axis=1)
#     files = [i for i in os.listdir(path) if i[:5] > max(data.Date)[-5:] and '2020' in i]
    
#     for i in files:
#         print('loading {}'.format(i))

#         try:
#             temp = process(path + i)
#         except ValueError:
#             print('New file has different format.')
        
#         assert(data.shape[1] == 10)
#         data = pd.concat([data, temp], ignore_index=True)

# print('There are total {} records.'.format(data.shape[0]))

# data.sort_values('Date').to_csv('covid_19_US.csv')

In [119]:
data.sample(10)

,FIPS,County,State,Date,Lat,Long,Confirmed,Deaths,Recovered,Active
23820,38065.0,Oliver,North Dakota,2020-03-29,NaN,-101.340616,0.0,0.0,0.0,0
21378,47107.0,McMinn,Tennessee,2020-03-28,NaN,-84.615207,3.0,0.0,0.0,0
31099,35047.0,San Miguel,New Mexico,2020-03-31,NaN,-104.816356,1.0,0.0,0.0,0
44376,51740.0,Portsmouth,Virginia,2020-04-06,NaN,-76.359716,27.0,0.0,0.0,0
831,0.0,Riverside County,CA,2020-03-08,33.9533,-117.396100,1.0,0.0,0.0,0
29608,38043.0,Kidder,North Dakota,2020-03-30,NaN,-99.775079,0.0,0.0,0.0,0
16590,16011.0,Bingham,Idaho,2020-03-26,NaN,-112.397844,2.0,0.0,0.0,0
2853,38003.0,Barnes,North Dakota,2020-03-22,NaN,-98.066060,0.0,0.0,0.0,0
5996,47153.0,Sequatchie,Tennessee,2020-03-23,NaN,-85.410221,0.0,0.0,0.0,0
6799,48027.0,Bell,Texas,2020-03-23,NaN,-97.478503,16.0,0.0,0.0,0


In [447]:
def ETL(file):

    # loading COVID-19 data and zipcode
    print('Processing {}'.format(file[-14:]))
    data = pd.read_csv(file)
    zipinfo = pd.read_csv('uszips.csv')[['county_fips', 'county_name', 'state_name', 'lat', 'lng']].drop_duplicates()
    zipinfo = zipinfo.groupby(['county_fips', 'county_name', 'state_name']).agg('mean').reset_index()

    # extract US records and features
    data = data.loc[data.Country_Region == 'US', ['FIPS', 'Admin2', 'Province_State', 'Confirmed', 'Deaths', 'Recovered', 'Active']]

    # rename features
    data.columns = ['FIPS', 'County', 'State', 'Confirmed', 'Deaths', 'Recovered', 'Active']

    # remove 'City' in the County columns to match zipcode table
    data.County = data.County.str.replace(' City', '', regex=False)

    # final output features
    feature = ['FIPS', 'County', 'State', 'lat', 'lng', 'Confirmed', 'Deaths', 'Recovered', 'Active']

    # extract non-missing records
    x = pd.merge(data, zipinfo, left_on = ['FIPS', 'County', 'State'],
                 right_on = ['county_fips', 'county_name', 'state_name'], how = 'inner')[feature]

    # correct wrong FIPS and fill in missing
    y = data[~data.FIPS.isin(x.FIPS)]
    z = pd.merge(y, zipinfo, left_on = ['County', 'State'], right_on = ['county_name', 'state_name'], how = 'inner')
    z.FIPS = z.county_fips
    z = z[feature]

    # merge two tables
    data = pd.concat([x, z], ignore_index = True).groupby(['FIPS', 'County', 'State']).agg('sum').reset_index()
    
    return data

def increment(x, y):
    temp = pd.merge(x, y[['FIPS', 'County', 'State', 'Confirmed', 'Deaths', 'Recovered', 'Active']],
                    on=['FIPS', 'County', 'State'], how='left')

    temp['Confirmed_new'] = temp.Confirmed_x - temp.Confirmed_y
    temp['Deaths_new'] = temp.Deaths_x - temp.Deaths_y
    temp['Confirmed'] = temp.Confirmed_x
    temp['Deaths'] = temp.Deaths_x
    temp = temp[['FIPS', 'County', 'State', 'lat', 'lng', 'Confirmed', 'Confirmed_new', 'Deaths', 'Deaths_new']]

    return temp

In [465]:
df = pd.read_csv('covid_19_US.csv')
df[df.Deaths_new <0]

,FIPS,County,State,lat,lng,Confirmed,Confirmed_new,Deaths,Deaths_new,Active
364,13029.0,Bryan,Georgia,32.070623,-81.471627,28,-1.0,1,-1.0,0
467,13245.0,Richmond,Georgia,33.394148,-82.051018,138,43.0,3,-1.0,0
693,18133.0,Putnam,Indiana,39.656884,-86.818813,32,1.0,2,-1.0,0
1081,26033.0,Chippewa,Michigan,46.268454,-84.520559,0,-1.0,0,-1.0,0
1609,37069.0,Franklin,North Carolina,36.047485,-78.350580,27,-6.0,0,-1.0,0
1718,39035.0,Cuyahoga,Ohio,41.455164,-81.665658,1113,61.0,22,-1.0,0
1784,39171.0,Williams,Ohio,41.566449,-84.574923,3,1.0,0,-1.0,0
2495,53063.0,Spokane,Washington,47.648015,-117.336270,251,0.0,14,-3.0,0


In [514]:
path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
file = path + '04-11-2020.csv'
data10 = ETL(file)
data10[data10.County == 'Autauga']

Processing 04-11-2020.csv


,FIPS,County,State,lat,lng,Confirmed,Deaths,Recovered,Active
0,1001.0,Autauga,Alabama,32.580766,-86.615497,19,1,0,0


In [482]:
diff = increment(data10, data9)
diff.shape

(2632, 9)

In [518]:
pop = pd.read_csv('us_county.csv')[['fips', 'population']]
temp = pd.merge(diff, pop, left_on = 'FIPS', right_on = 'fips', how = 'left').drop('fips', axis = 1)
temp.eval("Confirmed_rate = Confirmed / population", inplace =True)
temp.eval("Deaths_rate = Deaths / population", inplace =True)

In [527]:
temp.head()

,FIPS,County,State,lat,lng,Confirmed,Confirmed_new,Deaths,Deaths_new,Confirmed_rate,Deaths_rate
0,1001.0,Autauga,Alabama,32.580766,-86.615497,19,2.0,1,0.0,0.000344,0.000018
1,1003.0,Baldwin,Alabama,30.594986,-87.724403,66,7.0,1,0.0,0.000317,0.000005
2,1005.0,Barbour,Alabama,31.780406,-85.435020,9,0.0,0,0.0,0.000349,0.000000
3,1007.0,Bibb,Alabama,33.020930,-87.090535,13,2.0,0,0.0,0.000577,0.000000
4,1009.0,Blount,Alabama,33.924849,-86.599899,12,0.0,0,0.0,0.000208,0.000000
